In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from wall_analysis import parse_experiments
#exp_name = '2020_12_18_stepper'; appendix = ""; distance = 51
#exp_name = '2020_12_18_flying'; appendix="_new"; distance = 0
#exp_name = '2021_03_01_flying';
exp_name = '2021_04_30_hover';
fname = f'../experiments/{exp_name}/all_data.pkl'

try:
    df_total = pd.read_pickle(fname)
    print('read', fname)
except:
    answer = input('Run wall_analysis.py to parse experiments? (y/[n])') or 'n'
    if answer == 'y':
        df_total = parse_experiments(exp_name)
        pd.to_pickle(df_total, fname)
        print('saved', fname)

# Overall analysis

In [ ]:
import seaborn as sns
from plotting_tools import save_fig

min_time = 0
max_time = 12

max_dist = 50 # cm
for i, row in df_total.iterrows():
    print('starting position:', row.positions[0, :])
    positions_cm = row.positions[:, :3] * 100
    
    fig, axs = plt.subplots(1, 3) 
    fig.set_size_inches(10, 3.3)
    fig.suptitle(row.appendix, y=1.0)
    
    mask_time = (row.seconds > min_time) & (row.seconds < max_time)
    time = row.seconds[mask_time]
    
    #axs[0].plot(x=positions_cm[:, 0], y=positions_cm[:, 1], color=colors())
    sns.scatterplot(x=positions_cm[mask_time, 0], y=positions_cm[mask_time, 1], 
                    hue=time, ax=axs[0], linewidth=0, 
                    #size=positions_cm[:, 2],
                    palette='inferno')
    axs[0].set_xlabel('x [cm]')
    axs[0].set_ylabel('y [cm]')
    #axs[0].set_xlim(-max_dist/2, max_dist/2)
    #axs[0].set_ylim(-max_dist/2, max_dist/2)
    axs[0].axis('equal')
    axs[0].legend(loc='lower right', title='time [s]')

    axs[1].plot(time, positions_cm[mask_time, 0], label='x')
    axs[1].plot(time, positions_cm[mask_time, 1], label='y')
    axs[1].plot(time, positions_cm[mask_time, 2], label='z')
    axs[1].set_xlabel('time [s]')
    axs[1].set_ylabel('movement [cm]')
    axs[1].set_ylim(-max_dist, max_dist)
    axs[1].legend(loc='lower right')

    axs[2].plot(time, row.positions[mask_time, 3], label='yaw')
    axs[2].set_ylabel('yaw [deg]')
    axs[2].set_xlabel('time [s]')
    axs[2].set_ylim(-20, 20)
    axs[2].legend(loc='lower right')

    plt.tight_layout()
    #save_fig(fig, f'plots/{exp_name}{row.appendix}_movement.pdf')

In [ ]:
from frequency_analysis import add_spectrogram
from plotting_tools import pcolorfast_custom

df_total = df_total.assign(spectrogram=None)
df_total = df_total.apply(add_spectrogram, axis=1)

mic_idx = 0

#maxi = np.nanmax(np.concatenate([*dfs.spectrogram], axis=1))
for i_col, row in df_total.iterrows():
    all_frequencies = np.unique(row.frequencies_matrix)
    spec = row.spectrogram[:, mic_idx, :]
    print(spec.shape, spec.dtype)
    spec[spec == 0] = np.nan
    total = np.nanmean(np.abs(spec), axis=1)
    
    label = str(f"{row.appendix}").replace('_', '')
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_title(f'spectrogram of mic{mic_idx}, appendix {row.appendix}')
    pcolorfast_custom(ax, row.seconds, all_frequencies, np.abs(spec))
    ax.set_ylabel('frequency [Hz]')

# Extract frequency slices

In [ ]:
# choose the experiment to analyze
row = df_total.iloc[0]

In [ ]:
from calibration import get_calibration_function_median, get_calibration_function_dict

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
if "newbuzzer" in row.appendix:
    calib_function, calib_freq = get_calibration_function_median(
        exp_name="2021_02_25_wall", mic_type="audio_deck", ax=ax
    )
elif "test" in row.appendix:
    calib_function, calib_freq = get_calibration_function_median(
        "2021_04_30_stepper", "audio_deck", ax=ax
    )
else:
    calib_function, calib_freq = get_calibration_function_median(
        exp_name="2021_02_23_wall", mic_type="audio_deck", ax=ax
    )

In [ ]:
# spec_masked, freqs_masked = data_collector.fill_from_row()
from data_collector import DataCollector
from inference import get_probability_bayes

data_collector = DataCollector(exp_name=exp_name)

signals_f_all = row.stft
frequencies_all = row.frequencies_matrix
positions_all = row.positions

fig, ax = plt.subplots()
fig.set_size_inches(10, 5)

freqs_calib = np.linspace(np.min(frequencies_all), np.max(frequencies_all), 30)
f_calib_all = calib_function(freqs_calib)
for i_mic in range(f_calib.shape[0]):
    axs[0, i_mic].plot(freqs_calib, f_calib_all[i_mic], label=f"calib", ls=":")

for i in range(signals_f_all.shape[0]):
    signals_f = signals_f_all[i]
    frequencies = frequencies_all[i]
    data_collector.fill_from_signal(
        signals_f, frequencies, distance=positions_all[i, 1], time=row.seconds[i]
    )

    # also analyze the remaining data.
    if i < signals_f_all.shape[0] - 1:
        f_slice, freqs, stds, d_slice = data_collector.get_current_frequency_slice(
            n_required=16, verbose=False
        )
    else:
        f_slice, freqs, stds, d_slice = data_collector.get_current_frequency_slice(
            n_required=None, verbose=False
        )

    if f_slice is not None:
        fig, axs = plt.subplots(3, signals_f_all.shape[1], sharey='row')
        fig.set_size_inches(15, 10)
        fig.suptitle(f"up to {data_collector.latest_time:.1f}s", y=0.9)
        
        valid_idx = (freqs >= min(calib_function.x)) & (freqs <= max(calib_function.x))
            
        freqs = freqs[valid_idx]
        
        f_calib = calib_function(freqs)
        f_slice = f_slice[:, valid_idx]
        d_slice = d_slice[valid_idx]
        distance_corr = nominal_distance + (d_slice * 1e2)
        
        f_theo = get_freq_slice_theory(
            freqs, distance_cm=nominal_distance
        ).T  # n_mics x n_freqs
        f_theo_corr = get_freq_slice_theory(freqs, distance_cm=distance_corr).T

        print("new frequency slice!")
        for i_mic in range(f_slice.shape[0]):
            axs[0, i_mic].plot(freqs, f_slice[i_mic], label='measured')
            axs[0, i_mic].legend(loc="upper right")
            
            f_calibrated = f_slice[i_mic] / f_calib[i_mic]

            axs[1, i_mic].plot(
                freqs, f_calibrated, label='measured', ls='-'
            )
            axs[1, i_mic].plot(
                freqs, f_theo[i_mic], label='theo', color=f"C6", ls=':'
            )
            axs[1, i_mic].plot(
                freqs, f_theo_corr[i_mic], label='theo corr', color="C7", ls='-.'
            )
            axs[0, i_mic].set_title(f'mic{i_mic}')
            
            
            # get probability distribution
            distances_bayes, proba, diff_bayes = get_probability_bayes(
                f_calibrated,
                freqs, 
                mic_idx=0,
                distance_range=[20, 60],
                sigma=1.0
            )
            axs[2, i_mic].plot(
                diff_bayes, proba 
            )
            
        axs[1, i_mic].legend()
        ax.plot(distance_corr, label=label)
        ax.axhline(nominal_distance)
        
        break

ax.set_xlabel("time idx")
ax.set_ylabel("rel. movement [cm]")
ax.legend(loc="upper right")

In [ ]:
from dataset_parameters import kwargs_datasets
from simulation import get_df_theory

nominal_distance = 42 #in cm 
kwargs = kwargs_datasets[exp_name]["audio_deck"]

distances_grid = np.arange(20, 60)
freqs_theo = np.linspace(kwargs["min_freq"], kwargs["max_freq"], 100)
df_matrix_theo = get_df_theory(freqs_theo, distances_grid, mic_idx)

In [ ]:
from data_collector import DataCollector
from inference import get_probability_cost, get_probability_fft
from frequency_analysis import get_index_matrix
from plotting_tools import add_colorbar
from simulation import get_freq_slice_theory

if 'newbuzzer' in row.appendix: 
    calib_function = calib_function_new
    calib_freq = calib_freq_new
elif 'test1' in row.appendix:
    calib_function = calib_function_v3
    calib_freq = calib_freq_v3
else:
    calib_function = calib_function_old
    calib_freq = calib_freq_old

time_mask = (row.seconds > kwargs["min_time"]) & (row.seconds < kwargs["max_time"]) 
f_slice_theory = get_freq_slice_theory(freqs, distance_cm=nominal_distance)
f_slice_theory_corr = get_freq_slice_theory(freqs, distance_cm=distances_corr)

#f_slice, freqs, stds = data_collector.get_frequency_slice()
#f_slice, freqs, stds = data_collector.get_frequency_slice_fixed(frequencies=calib_freq)
f_slice = f_slice[mic_idx]
f_calib = calib_function(freqs)[mic_idx]

fig, ax_slices = plt.subplots()
ax_slices.errorbar(freqs, f_slice, label='raw', yerr=stds[mic_idx])
ax_slices.plot(freqs, f_calib, label='calibration curve')
ax_slices.legend()

fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
ax.plot(freqs, f_slice/f_calib, label='calibrated')
ax.plot(freqs, f_slice_theory, label='theory')
ax.plot(freqs, f_slice_theory_corr, label='theory, corrected')

In [ ]:
    ### extract varying slice ### 
    #index_matrix = get_index_matrix(spec_masked)
    #strongest_freqs = freqs_masked[index_matrix[0, :]]
    #f_slice_varying = [spec_masked[f_idx, mic_idx, idx] for idx, f_idx in enumerate(index_matrix[0, :])]
    relative_ds = -row.positions[time_mask, 1] * 100
    absolute_yaws = row.positions[time_mask, 3]
    distances_y = nominal_distance + relative_ds

    fig, axs = plt.subplots(2, 1)
    fig.set_size_inches(10, 5)
    f_slice_norm -= np.mean(f_slice_norm)
    f_slice_norm /= np.std(f_slice_norm)
    f_slice_varying -= np.mean(f_slice_varying)
    f_slice_varying /= np.std(f_slice_varying)
    f_slice_varying_theory -= np.mean(f_slice_theory)
    f_slice_varying_theory /= np.std(f_slice_theory)
    f_slice_varying_norm -= np.mean(f_slice_varying_norm)
    f_slice_varying_norm /= np.std(f_slice_varying_norm)

    axs[0].plot(seconds, f_slice_varying, label='experimental')
    axs[0].plot(seconds, f_slice_varying_norm, label='experimental calibrated')
    axs[0].plot(seconds, f_slice_varying_theory, label='theoretical')
    axs[0].legend()
    axs[0].grid(which='both')
    axs[1].plot(seconds, strongest_freqs)
    axs[1].grid(which='both')


    ##### algorithm #####
    #proba_cost = get_probability_cost(
    #    f_slice, freqs, distances_grid, mic_idx=mic_idx
    #)
    #proba_cost_norm = get_probability_cost(
    #    f_slice_norm, freqs, distances_grid, mic_idx=mic_idx
    #)
    proba_cost_mvmt = get_probability_cost(
        f_slice_varying, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
    )
    proba_cost_mvmt_norm = get_probability_cost(
        f_slice_varying_norm, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
    )
    proba_cost_mvmt_theory = get_probability_cost(
        f_slice_varying_theory, strongest_freqs, distances_grid, mic_idx=mic_idx, relative_ds=relative_ds, absolute_yaws=absolute_yaws
    )
    fig_cost, ax_proba_cost = plt.subplots()
    fig.set_size_inches(5, 5)
    #ax_proba_cost.plot(distances_grid, proba_cost, label=label)
    #ax_proba_cost.plot(distances_grid, proba_cost_norm, label=label + ' calibrated')
    ax_proba_cost.plot(distances_grid, proba_cost_mvmt, label=label)
    ax_proba_cost.plot(distances_grid, proba_cost_mvmt_norm, label=label + ' calibrated')
    ax_proba_cost.plot(distances_grid, proba_cost_mvmt_theory, label=label + ' theory')
    ax_proba_cost.axvline(nominal_distance, label=f'ground truth {label}', color='black')
    ax_proba_cost.legend(loc='upper left', bbox_to_anchor=[1.0, 1.0]) 
    ax_proba_cost.set_ylabel('probability')
    ax_proba_cost.set_xlabel('distance [cm]')
    ax_proba_cost.set_yscale('log')

    ### full df ###
    fig, ax = plt.subplots()
    ax.pcolorfast(distances_grid, frequencies, df_matrix_theo[:-1, :-1])
    ax.scatter(distances_y, strongest_freqs, color='C1', s=5)
    ax.axvline(nominal_distance, color='C1')
    ax.set_ylim(min(frequencies), max(frequencies))
    ax.set_title(f'DF of mic{mic_idx}, {label}')
    ax.set_xlabel('distance [cm]')
    ax.set_ylabel('frequency [Hz]')
    save_fig(fig, f'plots/{exp_name}{row.appendix}_df.pdf')

    ### small df ###
    #d_sorted = np.sort(distances_y)
    #f_sorted = np.sort(strongest_freqs)
    d_sorted = np.arange(min(distances_y), max(distances_y), step=0.1) #np.sort(distances_y)
    f_sorted = np.arange(min(strongest_freqs), max(strongest_freqs), step=50) #np.sort(strongest_freqs)
    df_small = get_df_theory(f_sorted, d_sorted, mic_idx)
    fig, axs = plt.subplots(1, 2, sharey=True)
    axs[0].pcolorfast(d_sorted, f_sorted, df_small[:-1, :-1])

    frequency_indices = np.argmin(np.abs(f_sorted[:, None] - strongest_freqs[None, :]), axis=0)
    #np.testing.assert_allclose(f_sorted[frequency_indices], strongest_freqs)
    distance_indices = np.argmin(np.abs(d_sorted[:, None] - distances_y[None, :]), axis=0)
    df_small[frequency_indices, distance_indices] = f_slice_varying
    axs[1].pcolorfast(d_sorted, f_sorted, df_small[:-1, :-1])

#ax_proba_fft.set_xlim(min(distances_grid), max(distances_grid))
#save_fig(fig_cost, f'plots/{exp_name}{appendix_plot}_cost.pdf')
ax_slices.legend(loc='upper left')